# data loading prototyping

This notebook is just a scratchpad for experimenting with how to load data from the ATL03 files

In [2]:
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import pandas as pd
from cftime import num2date, date2num, num2pydate
from datetime import datetime, timedelta
from atl_module.load_netcdf import get_beams
import pdal

In [3]:
globcolorpath = "../data/global_analysis/ATL03/processed_ATL03_20181103000920_05400108_005_01.nc"

ds = Dataset(globcolorpath)

In [4]:
ds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    geospatial_lon_min: 34.45744904367203
    level: L2
    identifier_product_doi: doi:10.5067/ATLAS/ATL03.005
    contributor_name: Thomas E Neumann (thomas.neumann@nasa.gov), Thorsten Markus (thorsten.markus@nasa.gov), Suneel Bhardwaj (suneel.bhardwaj@nasa.gov) David W Hancock III (david.w.hancock@nasa.gov)
    keywords: EARTH SCIENCE > CRYOSPHERE > SEA ICE > NONE > NONE > NONE > NONE
    Processing Parameters: This file was gernerated by the ICESat-2 Service version 1.0 which performed the following operations on ATL03_20181103000920_05400108_005_01.h5
Extracted the datasets named:
/Metadata/
/ancillary_data/atlas_sdp_gps_epoch/
/ancillary_data/control/
/ancillary_data/data_end_utc/
/ancillary_data/data_start_utc/
/ancillary_data/end_cycle/
/ancillary_data/end_delta_time/
/ancillary_data/end_geoseg/
/ancillary_data/end_gpssow/
/ancillary_data/end_gpsweek/
/ancillary_data/end_orbit/
/ancillary_data

In [12]:
jarkus_path = "/home/mlinds/wsl_data/transect.nc"

In [72]:
arraylist = []
with xr.open_dataset(jarkus_path) as ds:
    # get a dataframe of the 2021 data
    for indexnum in range(1, 18):
        subset = ds.where(ds.areacode == indexnum, drop=True)
        subset = (
            subset.altitude.to_dataframe()
            .dropna()
            .reset_index()
            .rename(columns={"lat": "Y", "lon": "X", "altitude": "Z", "time": "date"})
            .drop(columns=["alongshore", "cross_shore"])
        )
        subset["date"] = pd.DatetimeIndex(subset["date"]).year
        subset_array = subset.to_records(index=False)
        arraylist.append(subset_array)

In [74]:
concat_array = np.concatenate(arraylist)

(20876194,)

In [73]:
p = pdal.Filter.reprojection(in_srs="EPSG:4326", out_srs="EPSG:7415").pipeline() | pdal.Writer.las(
    f"../data/jarkus/jarkus_{indexnum}.laz"
)

p.execute()

0

In [ ]:
ncdfds.

In [2]:
pd_dict = {
    "qa_perc_signal_conf_ph_high_ocean": 25.57604792974856,
    "qa_perc_signal_conf_ph_high_land": 15.58688869980957,
    "qa_perc_signal_conf_ph_low_ocean": 11.3459701986225,
    "qa_perc_signal_conf_ph_low_land": 60.38438713275513,
    "qa_perc_signal_conf_ph_med_ocean": 35.24949184009902,
    "qa_perc_signal_conf_ph_med_land": 13.202344265772433,
    "qa_perc_surf_type_ocean": 7.7244733010033,
    "qa_perc_surf_type_land": 22.280336125472594,
    "qa_total_signal_conf_ph_high_ocean": 47940,
    "qa_total_signal_conf_ph_high_land": 65399,
    "qa_total_signal_conf_ph_low_ocean": 21267,
    "qa_total_signal_conf_ph_low_land": 253359,
    "qa_total_signal_conf_ph_med_ocean": 66072,
    "qa_total_signal_conf_ph_med_land": 55394,
}

pd.DataFrame(pd_dict, index=[0])

,qa_perc_signal_conf_ph_high_ocean,qa_perc_signal_conf_ph_high_land,qa_perc_signal_conf_ph_low_ocean,qa_perc_signal_conf_ph_low_land,qa_perc_signal_conf_ph_med_ocean,qa_perc_signal_conf_ph_med_land,qa_perc_surf_type_ocean,qa_perc_surf_type_land,qa_total_signal_conf_ph_high_ocean,qa_total_signal_conf_ph_high_land,qa_total_signal_conf_ph_low_ocean,qa_total_signal_conf_ph_low_land,qa_total_signal_conf_ph_med_ocean,qa_total_signal_conf_ph_med_land
0,25.576048,15.586889,11.34597,60.384387,35.249492,13.202344,7.724473,22.280336,47940,65399,21267,253359,66072,55394


In [21]:
h5filepath = "../data/test_sites/Clipperton/ATL03/processed_ATL03_20191031163249_05310507_005_01.nc"
# "C:\Users\maxli\OneDrive - Van Oord\Documents\thesis\data\ATL12_FL\239711552\processed_ATL12_20210613064553_12291101_005_02.nc"

In [22]:
print(get_beams(h5filepath))

['gt3l', 'gt3r']


In [23]:
ncds = Dataset(h5filepath)
for x, y in ncds.groups["ancillary_data"].variables.items():
    print(y[:])

1198800018.0
cf_name=CTL_atlas_l2a_alt_008007285.ctl atlas_l2a_alt
asas.atl03.rapid=0
asas.atl03.use_def_uncert=1
asas.atl03.def_sigma_lat=0.000063
asas.atl03.def_sigma_lon=7.0, 111000.0, 0.0, 0.0, 0.0, 0.0, 0.0
asas.atl03.def_sigma_along=5.0
asas.atl03.def_sigma_across=5.0
asas.atl03.def_sigma_h=0.17
IN_ANC01=GEOS.fpit.asm.inst3_3d_asm_Nv.GEOS5124.20191031_1200.V01.nc4 1256558418.0000 1256569218.0000 001 01 2380b3dd-c23c-34da-9d72-2765f32ed510
IN_ANC01=GEOS.fpit.asm.inst3_3d_asm_Nv.GEOS5124.20191031_1500.V01.nc4 1256569218.0000 1256580018.0000 001 01 fb78c049-35dd-31f3-b1ce-e1212740a3b1
IN_ANC01=GEOS.fpit.asm.inst3_3d_asm_Nv.GEOS5124.20191031_1800.V01.nc4 1256580018.0000 1256590818.0000 001 01 73805747-4696-333d-b172-f304e02fcc7f
IN_ANC03=ANC03_f_201930400000_002_v01.h5 1256506200.0000 1256610600.0000 006 01 79f48310-f6fa-3d6b-b80d-058fcbb30747
IN_ANC04=ANC04_f_201930400000_004_rpy_v01.h5 1256506200.0000 1256610600.0000 007 01 7f163a39-26e7-33dd-b686-3a5bbac8b1a1
IN_ANC05=ANC05_f_2019

In [24]:
ncds.groups["gt2l"].groups["heights"].variables["delta_time"]

KeyError: 'heights'

In [15]:
ma = ncds.groups["gt2l"].groups["geolocation"].variables["ref_azimuth"][:]
ma

KeyError: 'geolocation'

In [18]:
print(ncds.groups["gt3l"])

<class 'netCDF4._netCDF4.Group'>
group /gt3l:
    Description: Each group contains the segments for one Ground Track. As ICESat-2 orbits the earth, sequential transmit pulses illuminate six ground tracks on the surface of the earth.  The track width is approximately 14m.  Each ground track is numbered, according to the laser spot number that generates a given ground track.  Ground tracks are numbered from the left to the right in the direction of spacecraft travel as: 1L, 1R in the left-most pair of beams; 2L, 2R for the center pair of beams; and 3L, 3R for the right-most pair of beams.
    atlas_pce: pce1
    atlas_beam_type: weak
    groundtrack_id: gt3l
    atmosphere_profile: profile_3
    atlas_spot_number: 2
    sc_orientation: Forward
    dimensions(sizes): 
    variables(dimensions): 
    groups: bckgrd_atlas, signal_find_output


In [25]:
ncds.groups["gt3l"].groups["geophys_corr"].variables["tide_ocean"][:]

masked_array(data=[0.40286356, 0.4028628 , 0.40286198, 0.40286106,
                   0.40286028, 0.4028595 , 0.40285876, 0.40285796,
                   0.40285718, 0.40285638, 0.40285563, 0.40285477,
                   0.40285403, 0.40285322, 0.40285245, 0.40285164,
                   0.4028508 , 0.40284994, 0.4028492 , 0.40284833,
                   0.40284753, 0.40284666, 0.40284583, 0.402845  ,
                   0.40284413, 0.40284327, 0.40284237, 0.40284154,
                   0.4028407 , 0.4028398 , 0.402839  , 0.40283823,
                   0.40283746, 0.40283665, 0.40283585, 0.40283513,
                   0.40283433, 0.40283355, 0.40283275, 0.40283194,
                   0.40283108, 0.4028302 , 0.40282938, 0.40282857,
                   0.40282774, 0.40282694, 0.40282613, 0.4028254 ,
                   0.4028246 , 0.40282384, 0.40282306, 0.40282232,
                   0.40282154, 0.40282065, 0.4028199 , 0.40281916,
                   0.4028181 , 0.40281752, 0.40281665, 0.40281

In [9]:
"heights" in ncds.groups["gt1l"].nc

True

In [70]:
QA_data = {}

for varname, values in ncds.groups["quality_assessment"].groups["gt3l"].variables.items():
    QA_data[varname + "_ocean"] = values[:].data[0][1]
    QA_data[varname + "_land"] = values[:].data[0][0]
print(QA_data)

{'qa_perc_signal_conf_ph_high_ocean': 20.2397372931732, 'qa_perc_signal_conf_ph_high_land': 8.261628004495211, 'qa_perc_signal_conf_ph_low_ocean': 13.601657500014428, 'qa_perc_signal_conf_ph_low_land': 67.2428252738221, 'qa_perc_signal_conf_ph_med_ocean': 38.63902627645392, 'qa_perc_signal_conf_ph_med_land': 13.559156197409104, 'qa_perc_surf_type_ocean': 7.402508917478257, 'qa_perc_surf_type_land': 23.131337421345837, 'qa_total_signal_conf_ph_high_ocean': 35070, 'qa_total_signal_conf_ph_high_land': 29553, 'qa_total_signal_conf_ph_low_ocean': 23568, 'qa_total_signal_conf_ph_low_land': 240537, 'qa_total_signal_conf_ph_med_ocean': 66951, 'qa_total_signal_conf_ph_med_land': 48503}


In [80]:
existing_dict = {"testdict": 1}
QA_data.update(existing_dict)
print(QA_data)

{'qa_perc_signal_conf_ph_high_ocean': 20.2397372931732, 'qa_perc_signal_conf_ph_high_land': 8.261628004495211, 'qa_perc_signal_conf_ph_low_ocean': 13.601657500014428, 'qa_perc_signal_conf_ph_low_land': 67.2428252738221, 'qa_perc_signal_conf_ph_med_ocean': 38.63902627645392, 'qa_perc_signal_conf_ph_med_land': 13.559156197409104, 'qa_perc_surf_type_ocean': 7.402508917478257, 'qa_perc_surf_type_land': 23.131337421345837, 'qa_total_signal_conf_ph_high_ocean': 35070, 'qa_total_signal_conf_ph_high_land': 29553, 'qa_total_signal_conf_ph_low_ocean': 23568, 'qa_total_signal_conf_ph_low_land': 240537, 'qa_total_signal_conf_ph_med_ocean': 66951, 'qa_total_signal_conf_ph_med_land': 48503, 'testdict': 1}


In [54]:
[np.array(varname[:])]

[array([[ 8.261628  , 20.23973729,  0.        ,  0.        , 10.5309377 ]]),
 array([[67.24282527, 13.6016575 ,  0.        ,  0.        , 38.27739431]]),
 array([[13.5591562 , 38.63902628,  0.        ,  0.        , 38.35642904]]),
 array([[23.13133742,  7.40250892,  0.        ,  0.        ,  3.04129427]]),
 array([[29553, 35070,     0,     0,  5996]]),
 array([[240537,  23568,      0,      0,  21794]]),
 array([[48503, 66951,     0,     0, 21839]])]

In [21]:
with xr.open_dataset(h5filepath, group="gt1l/heights") as xrds:
    xrds

AttributeError: 'NoneType' object has no attribute 'dimensions'

In [14]:
xrds = xr.open_dataset(h5filepath, group="gt3r/geophys_corr")

xrds

<xarray.Dataset>
Dimensions:          (delta_time: 1420)
Coordinates:
  * delta_time       (delta_time) datetime64[ns] 2019-09-28T03:21:07.44542145...
Data variables:
    dem_h            (delta_time) float32 ...
    geoid            (delta_time) float32 ...
    geoid_free2mean  (delta_time) float32 ...
    tide_ocean       (delta_time) float32 ...
Attributes:
    Description:  Contains parameters used to correct photon heights for sele...
    data_rate:    These parameters are stored at the ICESat-2 Geolocation Alo...

In [15]:
z_correction = xrds.geoid_free2mean + xrds.geoid * -1 + xrds.tide_ocean
z_correction

<xarray.DataArray (delta_time: 1420)>
array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)
Coordinates:
  * delta_time  (delta_time) datetime64[ns] 2019-09-28T03:21:07.445421456 ......

In [15]:
st_rgt = Dataset(h5filepath).groups

# ['ancillary_data'].variables['start_rgt'][:]

In [31]:
xrds = xr.open_dataset(h5filepath, group="gt3l/geophys_corr")
xrds.delta_time.shape

(1284,)

In [40]:
a = ncds.groups["gt3l"].groups["heights"].variables["signal_conf_ph"][:, 1]
np.array(a)

array([4, 4, 4, ..., 4, 4, 4], dtype=int8)

In [95]:
ncds

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    geospatial_lon_min: -81.35265359722906
    level: L2
    identifier_product_doi: doi:10.5067/ATLAS/ATL03.005
    contributor_name: Thomas E Neumann (thomas.neumann@nasa.gov), Thorsten Markus (thorsten.markus@nasa.gov), Suneel Bhardwaj (suneel.bhardwaj@nasa.gov) David W Hancock III (david.w.hancock@nasa.gov)
    keywords: EARTH SCIENCE > CRYOSPHERE > SEA ICE > NONE > NONE > NONE > NONE
    Processing Parameters: This file was gernerated by the ICESat-2 Service version 1.0 which performed the following operations on ATL03_20201202073402_10560901_005_01.h5
Extracted the datasets named:
/Metadata/
/ancillary_data/data_end_utc/
/ancillary_data/data_start_utc/
/ancillary_data/end_rgt/
/ancillary_data/start_rgt/
/quality_assessment/qa_granule_pass_fail/
/gt1l/geolocation/segment_id/
/gt1l/geolocation/sigma_h/
/gt1l/geophys_corr/delta_time/
/gt1l/geophys_corr/dem_h/
/gt1l/geophys_corr/geoid/
/gt1l/geophys

In [99]:
ncds.groups["gt1l"].groups["heights"].ncattrs()

['Description', 'data_rate']

In [103]:
ncds.description

'This data set (ATL03) contains height above the WGS 84 ellipsoid (ITRF2014 reference frame), latitude, longitude, and time for all photons downlinked by the Advanced Topographic Laser Altimeter System (ATLAS) instrument on board the Ice, Cloud and land Elevation Satellite-2 (ICESat-2).'

In [4]:
a = np.random.randint(0, 100, size=100)
b = np.random.normal(size=100)
a

array([75, 95,  1, 37, 76, 67, 27, 58, 75, 21, 16, 13, 35, 52, 49, 27, 31,
        5, 91, 76, 43, 82, 53, 82, 15, 66, 54, 13,  9, 61, 42, 98, 73, 13,
       57, 43, 43, 24, 52, 52, 65, 81, 85, 87,  9, 99, 95, 77, 49, 22, 10,
        8, 19, 53, 45, 18, 73, 44, 54,  1, 91, 40, 31, 33, 75, 26, 75, 78,
       41, 94, 23, 49, 89, 58, 74, 24, 84,  1, 89, 33, 30, 60,  5, 12,  9,
       33, 49, 68, 33, 85, 87, 56, 33, 87, 89, 36, 69, 91, 13, 57])

ERROR! Session/line number was not unique in database. History logging moved to new session 11


In [5]:
b

array([-0.31503933, -0.05620716,  2.42325741,  0.79477423, -2.10528555,
        1.07067168,  0.15873523, -1.23223445, -0.00848203, -0.74975272,
        0.34598842,  0.17813241,  0.14363086, -1.81393551,  0.85364658,
       -0.85540962, -1.22403159, -1.55932301, -0.82171545,  0.04725742,
        0.994554  ,  1.5668754 , -1.07671755,  0.33862805, -1.26887244,
        0.46925538, -0.04374728,  0.95065031, -0.50043123, -0.31495834,
        0.78901074, -0.51898357,  0.24886347,  2.31458535,  1.46723933,
       -0.33569775, -0.33512831, -1.3762444 ,  0.74343667,  0.28107771,
        0.37403211, -0.09868099,  0.10666268, -1.77198646, -0.49649269,
        0.03095494,  0.49872536, -0.20331527,  0.09356994,  1.62534325,
       -0.47315099,  0.49935274, -0.90913035,  0.58820077, -1.23216652,
       -0.1809294 ,  0.60483326, -0.18931249, -0.92608823, -0.54023788,
        0.15859822,  0.30248788,  2.54711181, -0.78851716, -1.99073433,
       -1.83204399,  0.2311412 , -0.06764748,  0.13356877, -0.33

In [ ]:
np.array((a, b), dtype=[("aname", "<i8"), ("bname", "<f8")])